# MLOPS

Snippet codes for deploy ML applications.


## Best practices

**TODO** Search more about good practices

1. You need to change enviroment just change configuration files or enviroment variables and NOT code. The code should be the same running on developemwnt, stagibg or production

1. Use enviroment variable
    * credentials. You can add this envriroment variable in the .env file on your home
    * databases or other resources addreess and etc

1. Share nothing archicteture (for easy scaling)

## Bash

For quick documenattion


```sh
tldr tar

```

* emacs package howdoyou


## AWS 

ref: udacity course material it is a goodreference

**TODO**

* List of main services what does it do. Waht is good for. Comomn mistakes and do not use it?

    1. S3
    1. RDS 
    1. Dynamodb
    1. Redshift  <=== for caching. Most recent date is caching, then migrate to postgresql db old data
    1. Kafka <== 
    1. EC2
    1. ECR
    1. EKS  <== 
    1. EMR  <===
    1. ROUTE53 
    1. VPC

* Main boring tasks like:

    * set IAM
    * set group permission
    * open port 

* Main terms

    * region
    * AZ
    * VPC
    * Install and set AWS CLI
    
* Analytics and ML

    * Athena
    * sagemaker

### S3
refs:
* https://docs.aws.amazon.com/cli/latest/reference/s3/rm.html
* https://stackoverflow.com/questions/51375531/aws-cli-s3-bucket-remove-object-with-date-condition

Preview commands 

```sh
aws s3 any_command --dryrun

```

Clean S3 bucket fastest way

```sh
aws s3 rm s3://bucket-dev --recursive

# exclude pattern
aws s3 rm s3://mybucket/ --recursive --exclude "*.jpg"

```

**Using python script**

```python
import boto3    
s3 = boto3.resource('s3')
bucket = s3.Bucket('my-bucket')
# suggested by Jordon Philips 
bucket.objects.all().delete()
```

Useful commands
```sh

# list files in the buckets
aws s3 ls s3://bucket-dev

# delete key
aws s3 rm s3://bucket-dev/test2.txt

# delete all txt
aws s3 rm s3://bucket-dev/test-folder/ --include "*.txt"

# count number of files
aws s3 ls --recursive s3://bucket-dev/ | wc -l
```

**delete multiple files based on pattern**
<p style="color:red"># BECAREFUL: THIS COMMAND DO NOT ASK TO DELETE BEFORE DELETE</p>

```sh

# 1) filter the files based on pattern match using grep
aws s3 ls s3://bucket-dev/ | awk '{print $4}' | grep -E -i 00a5b462ef48280440ecc6af98133d77_leandro_test2


# 2) Pass the output to xargs
aws s3 ls s3://bucket-dev/ | awk '{print $4}' | grep -E -i 00a5b462ef48280440ecc6af98133d77_leandro_test2 | xargs -I% echo %_TEXTTTT


# 3) Use xrags to run aws s3 rm key
aws s3 ls s3://bucket-dev/ | awk '{print $4}' | grep -E -i 00a5b462ef48280440ecc6af98133d77_leandro_test2 | xargs -I% bash -c 'aws s3 rm s3://bucket-dev/%'

```

**delete mutliple files based on date condition**
<p style="color:red"># BECAREFUL: THIS COMMAND DO NOT ASK TO DELETE BEFORE DELETE</p>

```sh

# 1) count the number of files first
aws s3 ls --recursive s3://bucket-dev/ | awk '$1 > "2021-01-20" && $1 < "2021-02-01" {print $4}'|  sort -n | wc -l

# 2) Use xargs to delete. xrags explanation: -n1 mean only one arg per line -t means verbose
aws s3 ls --recursive s3://bucket-dev/ | awk '$1 > "2021-01-20" && $1 < "2021-02-01" {print $4}'|  sort -n | xargs -I% bash -c 'aws s3 rm s3://bucket-dev/%'

```


## Docker

### Main terms

Maybe add some FAQ or doubts


### docker

What does he do?


### docker-compose 


What does he do?

### docker-machine 


What does he do?


## Kuebernetes

### Summary of main terms

* pods 

* deployment 

* services


### kubectl commands


```bash

# get info all services, replicas and deploymenst
kubectl get all

# get a list of all services running in the cluser
kubectl get svc

# describe service
kubectl describe services my-flask-service

# get info about pods
kubectl get pods  -o wide --show-labels

# get info about clusters nodes (nodes is different form pods)
kubectl get nodes

# "ssh" into a pod: kubectl exec -it <POD-NAME> /bin/sh
kubectl exec -it my-flask-844778465d-q4vwr /bin/sh # run ps, top to inspect the pod env

```



### eksctl commands

refs:
* https://thanhtungvo.medium.com/eksctl-a-great-cli-tool-for-amazon-eks-36636e268d90
* https://eksctl.io/introduction/


<p style="color:red">eksctl can RESTART EC2 INSTANCES AUTOMATIC. SO ALWAYS DELETE THE CLUSTER TO MAKE SURE NO INSTANCE WILL BE RUNNING ON AWS.</p>


```bash

# create cluster
eksctl create cluster --name=cluster-5 --nodes-min=1 --nodes-max=2

# get a list of clusters
eksctl get clusters

# delete cluster
eksctl delete cluster --name=<name> [--region=<region>]

# For ekstcl it is not automatic delete load balancer if there are any services in the clusters so you need to delete service by manual.
kubectl get svc --all-namespaces
kubectl delete svc service-name --namespace=[namesapce]



# Aloow ssh access to a node
eksctl create cluster --ssh-access --ssh-public-key=my_eks_node_id.pub
# sshould see a msg:
# importing SSH public key "/home/<USER_NAME>/.ssh/id_rsa.pub" as "eksctl-xxx-nodegroup-ng-axxx-xx:xx:xx:xx:xx:xx:xx"

# now you can ssh with:
ssh -i /home/<USER_NAME>/.ssh/id_rsa.pub ec2-user@ec2-xx-xx-xx-xx

# or to use pre-existing EC2 key pair in us-east-1 region, 
eksctl create cluster --ssh-access --ssh-public-key=my_kubernetes_key --region=us-east-1

```

### Example simple flask app and Load Balancer kubernetes service


> * You need AWS account ready
> * You need AWS CLI already ready with credentials set


#### Flask App

##### Create virtual cluster on AWS

The commmands takes about 5 to 10 minutes on AWS i order to get the cluster ready.

```bash
# Create a cluster with 2 nodes
# Needs permission to create IAM role for the cluster
eksctl create cluster \
       --name test-cluster \
       --region us-west-2 \
       --nodegroup-name linux-nodes \
       --node-type t2.micro \
       --nodes 2 # 2 work nodes

# Or you can save the conf in yaml file and run
eksctl create cluster -f cluster.yaml

```

Using yaml file: eks.yaml

```yaml
apiVersion: eksctl.io/v1alpha5
kind: ClusterConfig
metadata:
  name: my-cluster
  region: ap-southeast-1
managedNodeGroups:
  - name: job-manager
    instanceType: t3a.micro
    desiredCapacity: 2
    minSize: 2
    maxSize: 10
    tags:
      'Project': 'KubernetesNoob'
```

```bash

# Or you can save the conf in yaml file and run
eksctl create cluster -f cluster.yaml

```

You can inspect the cluster


```bash
# where kube save info about the cluster
cat ~/.kube/config

# you can run kubctl commands now
kubectl get nodes

# get namespace.
# kubectl support many virtual clsuter under the same physical cluster
kubectl get ns  
kubectl get pod ## It is empty now


# in case you want to delete
# Cleanup malformed clusters
eksctl delete cluster --region=us-west-2 --name=test-cluster
```


##### Push a dockerize app image to ECR

1. **Build the dockerized app**

```bash
docker build -f Dockerfile -t my-flask-image:latest

# see the image
docker images

# run the the dockerized app
docker run -p 5001:5000 my-flask-image

# In the case you want to run without docker
# http://localhost:5000 
python app/main.py
```

1. Create a new repo on ECR
    1. pickup a name: my-flask
    1. Look view push commands for help


```bash
export AWS_ACCOUNT_ID="44XXXXXXXX69"
```


```bash
# tag
docker tag my-flask-image:latest ${AWS_ACCOUNT_ID}.dkr.ecr.us-west-2.amazonaws.com/my-flask:latest

# get ECR authentication for docker client
aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin ${AWS_ACCOUNT_ID}.dkr.ecr.us-west-2.amazonaws.com

# push to ECR
docker push ${AWS_ACCOUNT_ID}.dkr.ecr.us-west-2.amazonaws.com/my-flask:latest

```

##### Create kuberbetes Load Balancer service


1. Create a Load Balnace service: Create the file: deployment.yaml (It has service and deployment kubectl configuration)
    
   * Expose the cluster PORT 8080 for external access
   * Instanciate 2 pods in the eks cluster. replicas of the dockerized flask app 
 
```yaml
apiVersion: v1
kind: Service   # <= service config
metadata:
  name: my-flask-service
spec:
  selector:
    app: my-flask
  ports:
  - protocol: "TCP"
    port: 6000    # <= service expose port in the cluster
    targetPort: 5000 # <== Target port. It port 6000 will be redirect to 5000
  type: LoadBalancer  # <= service type
---
apiVersion: apps/v1
kind: Deployment  # <= service config
metadata:
  name: my-flask
spec:
  selector:
    matchLabels:
      app: my-flask
  replicas: 2  # <= We are creating 2 replicas of flask app
  template:
    metadata:
      labels:
        app: my-flask
    spec:
      containers:
      - name: my-flask
        image: ${AWS_ACCOUNT_ID}.dkr.ecr.us-west-2.amazonaws.com/my-flask:latest   # <= ECR URI
        ports:
        - containerPort: 5000   # <= opened port in the containers 

```


Run the commands

```bash

kubectl apply -f deployment.yaml

# get info about the deployment 
kubectl get deployments my-flask

kubectl describe deployments my-flask

kubectl get replicasets
kubectl describe replicasets

```


#### Accessing the running container

**From local you can use kubectl proxy**


```bash
kubectl proxy --port 8080

# Get the API versions:
curl http://localhost:8080/api/

# Get list of pods
curl http://localhost:8080/api/v1/namespaces/default/pods

```

http://localhost:8080/api/v1/namespaces/default/services/my-flask-service/proxy/uname


> Every time you refresh you can see the Load Balancer calling different pod every time.



**From external**

1. Change the service expose port from 6000 to 8080. The port 6000 should NOT be opened on AWS.
1. Delete the service and recreate it again


```bash
# delete the service
kubectl delete svc  my-flask-service

# recreate again the service
kubectl apply -f deployment.yaml

# Get info about the service
kubectl describe services my-flask-service

```
Expect output of kubectl describe services 


```yaml
Name:                     my-flask-service
Namespace:                default
Labels:                   <none>
Annotations:              <none>
Selector:                 app=my-flask
Type:                     LoadBalancer
IP Families:              <none>
IP:                       <CLUSTER IP>
IPs:                      <none>
LoadBalancer Ingress:     <EXTERNAL IP or DNS>
Port:                     <unset>  8080/TCP
TargetPort:               5000/TCP
NodePort:                 <unset>  31228/TCP
Endpoints:                192.X.X.15X:5000,192.X.X.4X:5000
Session Affinity:         None
External Traffic Policy:  Cluster
Events:
  Type    Reason                Age   From                Message
  ----    ------                ----  ----                -------
  Normal  EnsuringLoadBalancer  42s   service-controller  Ensuring load balancer
  Normal  EnsuredLoadBalancer   40s   service-controller  Ensured load balancer

```

Notes:

* ```<EXTERNAL IP or DNS>``` : You should use this in the browser to access the swagger API of the app
* Endpoints: Are the IP address of each POD replica in the cluster under this service

http://```<EXTERNAL IP or DNS>```:8080/

### Kubernetes dashboards

## Jenkins 